In [1]:
%%bash
# Dowload the SQuAD1.1 data
curl -O https://data.deepai.org/squad1.1.zip
unzip -o squad1.1.zip -d data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8937k  100 8937k    0     0  3492k      0  0:00:02  0:00:02 --:--:-- 3491k


Archive:  squad1.1.zip
  inflating: data/dev-v1.1.json      
  inflating: data/train-v1.1.json    


In [2]:
# %%bash
#####################################################################
# Only use on Googgle Colab, uncomment if necessary
#####################################################################
# Clone the repo content into
# cd /content
# rm -rf deepqa
# git clone -b data_analysis https://github.com/PaulBeuran/deepqa.git

In [3]:
#####################################################################
# Only use on Googgle Colab, uncomment if necessary
#####################################################################
#%cd deepqa/notebooks/

In [4]:
import sys
import os
import pathlib

import json

import torch
import transformers

if torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')

deepqa_lib_path = str(pathlib.Path(os.getcwd()).parent.parent.absolute())
sys.path.insert(0, deepqa_lib_path)

from deepqa import preprocessing, tokenizer, model, loss, metrics

2022-07-29 10:46:39.758225: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-29 10:46:39.758300: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [5]:
# Read train and dev data
with open("data/train-v1.1.json", "rb") as j:
    train_data = json.load(j)["data"]
with open("data/dev-v1.1.json", "rb") as j:
    dev_data = json.load(j)["data"]

In [6]:
tokenizer_path = "bert-base-uncased"
context_max_length = 512
query_max_length = 64

contexts_token_ids, queries_token_ids, answers_token_ranges = preprocessing.tokenize_squad_11_data(
    train_data, tokenizer.HFAutoTokenizer("bert-base-uncased"), 512, 64
)

train_dataset = preprocessing.QADataset(contexts_token_ids, queries_token_ids, answers_token_ranges)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8)

In [ ]:
word_encoder = transfo